# ML Model Management with JFrog Artifactory and Security scanning (Xray)

## Install required PyPI packages from JFrog Artifactory

In [ ]:
# Configure ~/.pip/pip.conf as described in the JFrog Platform Set-Me-Up for the PyPI remote or virtual repository
%pip install ipywidgets matplotlib numpy pandas_datareader pillow pyyaml scikit-image seaborn torch torchvision transformers yfinance    

## Setup Hugging Face environment

In [ ]:
import yaml

# Read configuration file
with open("config.yml", 'r') as ymlfile: cfg = yaml.safe_load(ymlfile)
rt_ml_repo_url = cfg['rt']['ml']['repo_url']
rt_ml_access_token = cfg['rt']['ml']['access_token']

# Update the config.yml file with the Hugging Face repository URL and access token, as described on the Artifactory Set-Me-Up
%env HF_ENDPOINT=$rt_ml_repo_url
%env HUGGING_FACE_HUB_TOKEN=$rt_ml_access_token

## Import required packages

In [ ]:
# Update with the details of the company you want to analyse
stock_ticker = "AAPL"
stock_name = "Apple"

# Import required packages
import matplotlib.pyplot as plt, numpy as np, pandas as pd, seaborn as sns, skimage, torch, warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pandas_datareader import data as pdr
from PIL import Image, ImageDraw, ImageFont
from transformers import pipeline, AutoProcessor, AutoModelForZeroShotObjectDetection, AutoTokenizer, AutoModelForSequenceClassification
from transformers.utils import logging
from utilities import financial_dataset, read_news, merge_fin_news, sentim_analyzer, merge_dates

# Define print function for Makrdown syntax
from IPython.display import Markdown, display
def printmd(string): display(Markdown(string))

# Ignore warnings
logging.set_verbosity_error()

---

## Block unlicensed Hugging Face model with JFrog Security (Xray)
Prior to running this stage, create an Xray policy that blocks the download of artifacts with unknown licences in your ML Model Management repository and a watch to enforce the check. For more information, see [Creating Xray Policies and Rules](https://jfrog.com/help/r/jfrog-security-documentation/creating-xray-policies-and-rules). 

With the environment set up, we can start coding. We want to start using a Sentiment Analysis algorithm first.\
The model called **beto-sentiment-analysis** seems very popular and there are many articles about it, so we're curious to try it out.

In [ ]:
# Snippet to resolve Hugging Face packages via Artifactory, more details on the Artifactory Set-Me-Up
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError

try:
    snapshot_download(repo_id="finiteautomata/beto-sentiment-analysis", etag_timeout=1500000000)
except HfHubHTTPError as e:
    print("\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n")

Apparently, the model has no license definition and the JFrog policy we defined forbids the usage of unlicensed assets. We needed a new idea!

## Use Hugging Face model with Artifactory

There is an interesting image processing technology called **zero-shot-object-detection**. After doing some research, we found an article and a reference to a recommended model, which worked great and had an approved license. With the assistance of the model’s examples page, we wrote some code to use the model and identify frogs.

### Load and configure OWL-ViT Transformer

In [ ]:
# Snippet to resolve Hugging Face packages via Artifactory, more details on the Artifactory Set-Me-Up
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError

try:
    snapshot_download(repo_id="google/owlvit-base-patch32", etag_timeout=1500000000)
except HfHubHTTPError as e:
    print("\n\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n\n")

In [ ]:
checkpoint = "google/owlvit-base-patch32"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")
model = AutoModelForZeroShotObjectDetection.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

### Display Base Image

In [ ]:
im = Image.open("./img/frog2.jpg")
display(im)

### Where Are the Frogs?

Then, we add some more code to identify the objects and draw a marking square around them with the needed classification 

In [ ]:
text_queries = ["Frog", "Cat"]
inputs = processor(text=text_queries, images=im, return_tensors="pt")
draw = ImageDraw.Draw(im)

with torch.no_grad():
    outputs = model(**inputs)
    target_sizes = torch.tensor([im.size[::-1]])
    results = processor.post_process_object_detection(outputs, threshold=0.25, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(im)

scores = results["scores"].tolist()
labels = results["labels"].tolist()
boxes = results["boxes"].tolist()

for box, score, label in zip(boxes, scores, labels): 
    outline = "green"
    caption = text_queries[label]
    if label != 0: # Frog is the zero ordinal in lables
        outline = "red"
        caption = "Not a Frog!!!!"
        
    xmin, ymin, xmax, ymax = box
    font = ImageFont.truetype(r'fonts/OpenSans-VariableFont_wdth,wght.ttf', 60) 
    draw.rectangle((xmin, ymin, xmax, ymax), outline=outline, width=3)
    draw.text((xmin, ymin), f"{caption}", fill=outline, font = font)

display(im)
# Close image
im.close()

As you can see from this hypothetical (yet totally plausible) example, setting up JFrog to manage your ML models and integrate with your data scientists and ML Engineers flow is as simple as writing a few lines of code, but it immensely benefits an organization from a visibility and integrity perspective.

---

## Let's try something else ...

### Load Hugging Face Model for Sentiment Analysis with Artifactory

In [ ]:
# Snippet to resolve Hugging Face packages via Artifactory, more details on the Artifactory Set-Me-Up
from huggingface_hub import snapshot_download
from huggingface_hub.utils import HfHubHTTPError

try:
    snapshot_download(repo_id="ProsusAI/finbert", etag_timeout=1500000000)
except HfHubHTTPError as e:
    print("\n\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8 Xray blocked model download due to violation of the 'Block-Unknown' license policy.\U0001F6A8\U0001F6A8\U0001F6A8\U0001F6A8\n")

### Read and store headlines
The training data consists of 4M articles with daily financial news for 6000 stocks from 2009 to 2020

In [ ]:
# Read dataset
stock_news = read_news(stock_ticker)
stock_fin = financial_dataset(stock_ticker, num_of_labels = 2)
merged_stock = merge_fin_news(stock_fin, stock_news)

In [ ]:
# Create a tokenizer object
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Fetch the pretrained model 
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

### Run Sentiment Analysis (transformer inference)
We want to evaluate stock news and Sentiment Analysis with Fin-BERT Transformer. Sentiment analysis is a natural language processing technique of analyzing the emotion or positiveness of a sentence, in this case in relation to the impact that the news has on the company stock price.

In [ ]:
# Run Sentiment Analysis on the dataset
train_stock_df = sentim_analyzer(merged_stock, tokenizer, model)
pd.options.display.max_colwidth = 180

### Top Positive headlines

In [ ]:
train_stock_df[['date', 'headline', 'Positive']].sort_values('Positive',ascending = False).head(15)

### Top Negative headlines

In [ ]:
train_stock_df[['date', 'headline', 'Negative']].sort_values('Negative',ascending = False).head(15)

### Cleanup data and plot diagrams

In [ ]:
# Merge dates by computing average sentiment score for each date
train_stock_df_merged = merge_dates(train_stock_df)

# Store results into .csv file
#train_stock_df_merged.to_csv('Financial_News/trained/train_'+stock_ticker+'DB_labels.csv')

# Cleanup data
train_stock_df_merged['date'].unique()
data = train_stock_df_merged

# Calculate pie chart diagram metrics
incr = len(data[data['Price_change']==1])
decr = data.shape[0] - incr
positive = len(data[data['Positive']>(data['Negative']+data['Neutral'])])
negative = len(data[data['Negative']>(data['Positive']+data['Neutral'])])
neutral = len(data)-(positive+negative)
incr_ratio = (100 * positive) / data.shape[0]
decr_ratio = (100 * negative) / data.shape[0]

# Plot pie chart diagram and legend
fig, ax = plt.subplots(figsize=(6,6))
plt.pie(x=[positive, negative, neutral], labels=['Positive','Negative','Neutral'], shadow=True, autopct='%1.1f%%')
plt.title('Sentiment Analysis ', fontsize=18)
plt.show()
printmd("### **Total:** "+str(len(data))+"&emsp; **Positive:** "+str(positive)+"&emsp; **Negative:**&nbsp; "+str(negative)+"&emsp; **Neutral:** "+str(neutral))

In [ ]:
# Plot data distribution
fig , ax = plt.subplots(figsize=(16,10))
fig.suptitle("Data points distribution", fontsize=18)
sns.scatterplot(data=data, x="Positive", y="Negative", size='Neutral', hue = 'Price_change')
plt.show()

### Do we want to try with a new headline?

In [ ]:
# Define future headlines
headlines = [
    stock_name+" adopts JFrog Advanced Security and Curation to increase productivity and focus on the security threats that are impacting the business.",
    stock_name+" acquires JFrog for $10 billion deal that will enable the company to offer market-leading solutions to their end users.",
    stock_name+" margins shrink due to prolonged challenging market conditions and inflation.",
    "JFrog acquires "+stock_name+" for $5 result of a bet between the two CEOs after too many drinks.",
    stock_name+" pledge to become carbon-neutral by the end of this decade in response to popular demand.",
    stock_name+" revealed a new strategy to bring back employees to the office 4 days a week to pre-pandemic levels."
]

# Evaluate future headlines
printmd("## Breaking News from the future:")
for h in headlines:
    # Pre-process input phrase
    input = tokenizer(h, padding = True, truncation = True, return_tensors='pt')
    # Run inference on the tokenized phrase
    output = model(**input)
    # Pass model output logits through a softmax layer.
    sentim_scores = torch.nn.functional.softmax(output.logits, dim=-1)
    
    if sentim_scores[0][0] > (sentim_scores[0][1] + sentim_scores[0][2]):
        printmd("### \U0001F680 "+h)
    elif sentim_scores[0][1] > (sentim_scores[0][0] + sentim_scores[0][2]):
        printmd("### \U0001F62D "+h)
    else:
        printmd("### \U0001FAE4 "+h)